### TO DO:
(Problem statement)
(Describe the real-world phenomenon you are studying. Explain why the topic is important,
relevant, or interesting.)
(Uygar)

### TO DO:
(Data description)
Provide a brief description of your dataset, including the data source. 
(Uygar)

### Importing necessary libraries:

In [215]:
import pandas as pd 
import os 
import pymc


# Data preprocessing

### Load the data and put them into data dictionary 

In [216]:
# Data path
data_path = os.path.join(os.getcwd(), '..', 'Data')

# Load the data
life_expectancy = pd.read_csv(os.path.join(data_path, 'estat_demo_mlexpec_en.csv'))
material_deprivation_rate = pd.read_csv(os.path.join(data_path, 'estat_ilc_mdsd04_filtered_en.csv'))
median_and_mean_group_income = pd.read_csv(os.path.join(data_path, 'estat_ilc_di15_filtered_en.csv'))
low_work_intensity_households = pd.read_csv(os.path.join(data_path, 'estat_ilc_lvhl16n_filtered_en.csv'))
real_gdp = pd.read_csv(os.path.join(data_path, 'estat_tipsna40_en.csv'))
population_data = pd.read_csv(os.path.join(data_path, 'estat_demo_pjanbroad_filtered_en.csv'))


# Dataframes dictionary 
dataframes_dict = {
    'life_expectancy': life_expectancy,
    'material_deprivation_rate': material_deprivation_rate,
    'median_and_mean_group_income': median_and_mean_group_income,
    'low_work_intensity_households': low_work_intensity_households,
    'real_gdp': real_gdp,
    'population_data': population_data
}


### Include only EU data and set data collection window to 2003 - 2023
GDP data is not older than 2003, therefore we use a data collection window of 2003 - 2023. For some datasets we don't have data from 2024.

In [217]:
# Include only EU data and set data collection window
eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'Germany',
       'Denmark','Estonia',
       'Greece', 'Spain', 'Finland', 'France', 'Croatia', 'Hungary', 'Ireland', 'Italy',
       'Lithuania', 'Luxembourg', 'Latvia', 'Malta', 'Netherlands',
       'Poland', 'Portugal', 'Romania', 'Sweden', 'Slovenia', 'Slovakia']

for key, df in dataframes_dict.items():
    dataframes_dict[key] = df[df['Geopolitical entity (reporting)'].isin(eu_countries)].reset_index(drop=True)
for key, df in dataframes_dict.items():
    dataframes_dict[key] = df.loc[(df['TIME_PERIOD'] >= 2014) & (df['TIME_PERIOD'] < 2024)].reset_index(drop=True)


### Filter data for "Less than one year" age group
This data represents the average life expectancy.

In [218]:
# Filter life expectancies for newborns
dataframes_dict['life_expectancy'] = dataframes_dict['life_expectancy'].loc[dataframes_dict['life_expectancy']['age'] == 'Y_LT1'].reset_index(drop=True)

# Filter PPS currency
Median and mean data are available in three formats Euro, regional currency and PPS (Purchasing Power Standrad), we decide to use PPS because it adjusts for price level differences across regions, providing a more accurate basis for cross-country comparisons.

In [219]:
# Filter PPS (Purchasing Power Standard) currency 
dataframes_dict['median_and_mean_group_income'] = dataframes_dict['median_and_mean_group_income'].loc[dataframes_dict['median_and_mean_group_income']['unit'] == 'PPS'].reset_index(drop=True)


### Generalize mean and median data 
Since mean and median values in our dataset are segmented by age groups, we gather corresponding population data and compute weighted averages. This allows us to estimate overall population-level statistics for our analysis. We also remove unnecessary columns. 

In [220]:
# Generalize mean and median data
# Filter age groups present in the data
age_groups = ['From 18 to 64 years', '65 years or over']
dataframes_dict['median_and_mean_group_income'] = dataframes_dict['median_and_mean_group_income'][dataframes_dict['median_and_mean_group_income']['Age class'].isin(age_groups)].reset_index(drop=True)

# Remove unnecessary columns
for key, df in dataframes_dict.items():
    dataframes_dict[key] = df.drop(columns=['STRUCTURE', 'STRUCTURE_NAME', 'STRUCTURE_ID', 'Confidentiality status (flag)', 'CONF_STATUS', 'Observation status (Flag) V2 structure', 'unit', 'Observation value', 'Time', 'geo', 'age', 'Sex',
                                            'Country of citizenship', 'citizen', 'Unit of measure', 'Income and living conditions indicator', 'freq', 'Time frequency', 'OBS_FLAG', 'National accounts indicator (ESA 2010)', 'na_item'], errors='ignore')
    
    # Because the income date is available only by population, we need to merge it with population data to be able to combine the date for the whole population
dataframes_dict['population_data'].rename(columns={'OBS_VALUE': 'population'}, inplace=True)
dataframes_dict['population_data'].replace({'Age class': {'From 15 to 64 years': 'From 18 to 64 years'}}, inplace=True)
dataframes_dict['population_data'] = dataframes_dict['population_data'].loc[dataframes_dict['population_data']['Age class'] != 'Less than 15 years'].reset_index(drop=True)

dataframes_dict['median_and_mean_group_income'] = pd.merge(dataframes_dict['median_and_mean_group_income'],
                  dataframes_dict['population_data'],
                  on=['sex','Geopolitical entity (reporting)', 'TIME_PERIOD', 'Age class'],
                  how='left') 

# We group the data to calculate the weighted average of OBS_VALUE by population for each group defined
group_cols = ['sex', 'Geopolitical entity (reporting)', 'TIME_PERIOD', 'indic_il']

def weighted_average(group_df):
    weighted_sum = (group_df['OBS_VALUE'] * group_df['population']).sum()
    total_population = group_df['population'].sum()
    
    if total_population > 0:
        weighted_avg = weighted_sum / total_population
    else:
        weighted_avg = 0 
    
    return pd.Series({
        'OBS_VALUE': weighted_avg
    })

# Transform median and mean into seperate columns
dataframes_dict['median_and_mean_group_income'] = dataframes_dict['median_and_mean_group_income'].groupby(group_cols).apply(weighted_average).reset_index()
dataframes_dict['life_expectancy'] = dataframes_dict['life_expectancy'].drop(columns='Age class')
dataframes_dict['material_deprivation_rate'] = dataframes_dict['material_deprivation_rate'].drop(columns='Age class')
dataframes_dict['low_work_intensity_households'] = dataframes_dict['low_work_intensity_households'].drop(columns='Age class')

dataframes_dict['median_and_mean_group_income']['indic_il']
dataframes_dict['median_and_mean_group_income'] = dataframes_dict['median_and_mean_group_income'].pivot(index=[col for col in dataframes_dict['median_and_mean_group_income'].columns if col not in ['indic_il', 'OBS_VALUE']], columns='indic_il', values='OBS_VALUE').reset_index()


/var/folders/y4/lhggv2xd4_g7wl5ry_nch6q40000gn/T/ipykernel_31801/745170445.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataframes_dict['median_and_mean_group_income'] = dataframes_dict['median_and_mean_group_income'].groupby(group_cols).apply(weighted_average).reset_index()


In [221]:
merged

,sex,Geopolitical entity (reporting),TIME_PERIOD,MED_E,MEI_E,OBS_VALUE,real_gdp,mean-median,qoutient
0,F,Austria,2014,23041.372948,25606.672200,84.0,39520,2565.299252,0.111334
1,F,Austria,2015,23263.169051,25342.505855,83.7,39640,2079.336804,0.089383
2,F,Austria,2016,24605.250521,26527.179513,84.1,39970,1921.928992,0.078111
3,F,Austria,2017,24844.221020,27311.593112,84.0,40620,2467.372093,0.099314
4,F,Austria,2018,24443.071028,26600.126648,84.1,41430,2157.055620,0.088248
...,...,...,...,...,...,...,...,...,...
805,T,Sweden,2019,21749.500013,23594.508859,83.2,47790,1845.008845,0.084830
806,T,Sweden,2020,21291.441958,22935.643461,82.4,46550,1644.201503,0.077224
807,T,Sweden,2021,21823.628519,23617.433935,83.1,48690,1793.805416,0.082196
808,T,Sweden,2022,21853.884176,23521.927826,83.1,48780,1668.043650,0.076327


# Rename columns

In [222]:
# Rename columns 
dataframes_dict['real_gdp'] = dataframes_dict['real_gdp'].rename(columns={'OBS_VALUE': 'real_gdp'})
dataframes_dict['low_work_intensity_households'] = dataframes_dict['low_work_intensity_households'].rename(columns={'OBS_VALUE': 'people_in_low_work_intensity_household_perc'})
dataframes_dict['material_deprivation_rate'] = dataframes_dict['material_deprivation_rate'].rename(columns={'OBS_VALUE': 'material_deprivation_rate_perc'})


In [223]:
dataframes_dict['low_work_intensity_households']


,c_birth,Country/region of birth,sex,Geopolitical entity (reporting),TIME_PERIOD,people_in_low_work_intensity_household_perc
0,NAT,Reporting country,F,Austria,2015,4.4
1,NAT,Reporting country,F,Austria,2016,5.3
2,NAT,Reporting country,F,Austria,2017,4.9
3,NAT,Reporting country,F,Austria,2018,4.7
4,NAT,Reporting country,F,Austria,2019,4.7
...,...,...,...,...,...,...
727,NAT,Reporting country,T,Slovakia,2019,6.1
728,NAT,Reporting country,T,Slovakia,2020,4.5
729,NAT,Reporting country,T,Slovakia,2021,4.9
730,NAT,Reporting country,T,Slovakia,2022,3.5


In [224]:
merged = pd.merge(dataframes_dict['median_and_mean_group_income'], dataframes_dict['life_expectancy'], on=['sex', 'TIME_PERIOD', 'Geopolitical entity (reporting)']).reset_index(drop=True)
merged = pd.merge(merged, dataframes_dict['real_gdp'], on=['TIME_PERIOD', 'Geopolitical entity (reporting)'])
pd.merge(merged, dataframes_dict['material_deprivation_rate'], on=['sex', 'TIME_PERIOD', 'Geopolitical entity (reporting)']).reset_index(drop=True)


merged['mean-median'] = merged['MEI_E'] - merged['MED_E']
merged['qoutient'] = (merged['MEI_E'] - merged['MED_E']) / merged['MED_E']
women = merged.loc[merged['sex'] == 'F'].reset_index(drop=True)
men = merged.loc[merged['sex'] == 'M'].reset_index(drop=True)
total = merged.loc[merged['sex'] == 'T'].reset_index(drop=True)

corr_matrix_women = women[['MED_E', 'MEI_E', 'OBS_VALUE', 'mean-median', 'qoutient', 'real_gdp']].corr()
corr_matrix_men = men[['MED_E', 'MEI_E', 'OBS_VALUE', 'mean-median', 'qoutient', 'real_gdp']].corr()
corr_matrix_total = total[['MED_E', 'MEI_E', 'OBS_VALUE', 'mean-median', 'qoutient', 'real_gdp']].corr()

print(corr_matrix_women)
print(corr_matrix_men)
print(corr_matrix_total)


                MED_E     MEI_E  OBS_VALUE  mean-median  qoutient  real_gdp
MED_E        1.000000  0.995950   0.668230     0.660977 -0.316581  0.872864
MEI_E        0.995950  1.000000   0.675476     0.725769 -0.236419  0.869458
OBS_VALUE    0.668230  0.675476   1.000000     0.524749 -0.204224  0.537165
mean-median  0.660977  0.725769   0.524749     1.000000  0.449093  0.578019
qoutient    -0.316581 -0.236419  -0.204224     0.449093  1.000000 -0.264373
real_gdp     0.872864  0.869458   0.537165     0.578019 -0.264373  1.000000
                MED_E     MEI_E  OBS_VALUE  mean-median  qoutient  real_gdp
MED_E        1.000000  0.995592   0.719632     0.644273 -0.278261  0.868714
MEI_E        0.995592  1.000000   0.717304     0.713166 -0.193164  0.863839
OBS_VALUE    0.719632  0.717304   1.000000     0.470518 -0.255935  0.628401
mean-median  0.644273  0.713166   0.470518     1.000000  0.504596  0.551168
qoutient    -0.278261 -0.193164  -0.255935     0.504596  1.000000 -0.239589
real_gdp    